In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 6 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

28 November 2022 ; 13:16:39


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,Precious Achiuwa,C,23,TOR,12,0,20.4,3.0,7.7,0.391,...,1.79,114.14,112.51,1.63,1.89,114.64,112.87,1.77,0,1
2,Dalano Banton,PG,23,TOR,15,2,11.2,2.3,5.5,0.427,...,1.79,114.14,112.51,1.63,1.89,114.64,112.87,1.77,0,1
4,Khem Birch,C,30,TOR,9,0,8.0,1.0,1.6,0.643,...,1.79,114.14,112.51,1.63,1.89,114.64,112.87,1.77,0,1
5,Chris Boucher,PF,30,TOR,15,0,23.8,4.5,9.5,0.476,...,1.79,114.14,112.51,1.63,1.89,114.64,112.87,1.77,0,1
6,Justin Champagnie,SF,21,TOR,2,0,4.5,1.5,1.5,1.000,...,1.79,114.14,112.51,1.63,1.89,114.64,112.87,1.77,0,1


In [5]:
# Set training data
train_data = train_data[train_data['Year'] < 2020] # DELETE THIS LINE
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Bobby Portis,27.0,19,5,MIL,18.2,2.157895,7.336842,13.5,2,1.6,3.021053,0.258295
1,Malcolm Brogdon,22.7,16,0,BOS,19.1,2.050000,5.125000,13.6,1,3.1,3.587500,0.184831
2,Bones Hyland,21.5,13,0,DEN,17.2,1.261538,3.153846,15.0,2,2.8,1.892308,0.173664
3,Alec Burks,21.2,9,0,DET,21.5,1.822222,6.377778,16.7,15,3.4,5.466667,0.159612
4,Kevin Love,20.7,16,1,CLE,18.8,3.587500,7.175000,10.6,3,4.6,4.100000,0.152908


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)